# Trim LabelStudio annotations

> "to match trimmed audio"

- branch: master
- hidden: true
- comments: false
- categories: [labelstudio, trim, csv]

In [2]:
def slurpfile(filename) -> str:
    with open(filename) as inf:
        return inf.read().strip()

In [3]:
output_dir = "/tmp/textgrid_cut"
host = "http://130.237.3.107:8080/api/"
api_token: str = slurpfile("label_studio_mine")

In [4]:
from pathlib import Path

output_path = Path(output_dir)

In [5]:
import requests
import json
from pathlib import Path

headers = {
    "Authorization": f"Token {api_token}"
}

In [6]:
def get_task(task_id):
    ep = f"{host}tasks/{task_id}"
    req = requests.get(ep, headers=headers)
    if req.status_code != 200:
        return {}
    data = json.loads(req.text)
    return data

In [7]:
def get_annotation(annot_it):
    ep = f"{host}annotations/{annot_it}"
    req = requests.get(ep, headers=headers)
    assert req.status_code == 200
    data = json.loads(req.text)
    return data

In [8]:
data = get_annotation(264)

In [10]:
def combine_labels(data):
    combined = {}

    if "result" in data:
        for res in data["result"]:
            if not res["id"] in combined:
                combined[res["id"]] = res
            else:
                if "text" in res["value"]:
                    combined[res["id"]]["value"]["text"] = res["value"]["text"]
                elif "labels" in res["value"]:
                    combined[res["id"]]["value"]["labels"] = res["value"]["labels"]
    return combined

In [11]:
combined = combine_labels(data)

In [58]:
def adjust_times_write_tsv(data):
    task = data["task"]
    task_data = get_task(task)
    if "data" in task_data and "audio" in task_data["data"]:
        orig_file = task_data["data"]["audio"]
        parts = orig_file.split("/")
        orig_file = parts[-1]
    if orig_file:
        out_part = orig_file.replace(".wav", ".tsv")
        orig_file = out_part.replace("_main", "").replace("_inter", "")
    else:
        return []
    out_file = output_path / out_part

    results = []

    combined = combine_labels(data)

    for item in combined:
        val = combined[item]["value"]
        if not "labels" in val:
            continue
        if not "Speech" in val["labels"]:
            continue
        text = val["text"]
        if len(text) > 1:
            for t in text:
                if not (t.startswith("/") and t.endswith("/")):
                    text = t
        else:
            text = text[0]

    with open(out_file, "w") as outf:
        for res in results:
            outf.write("\t".join([str(x) for x in list(res)]) + "\n")
        